# **Notebook 2 – Model Training & Validation**

This notebook focuses on building, training, validating, and evaluating machine learning models to predict **pitch** using wind turbine sensor data.  
It includes data loading, preprocessing, train/validation/test splitting, model training, hyperparameter tuning, metrics evaluation, and final model export.

---

## 📑 **Table of Contents**

1. [Imports & Setup](#imports--setup)
2. [Load & Inspect Data](#load--inspect-data)
3. [Feature Selection](#feature-selection)
4. [Train/Validation/Test Split](#trainvalidationtest-split)
5. [Data Scaling](#data-scaling)
6. [Model Training](#model-training)
7. [Model Evaluation](#model-evaluation)
8. [Save Model & Scaler](#save-model--scaler)
9. [Hyperparameter Tuning](#hyperparameter-tuning)
10. [Final Model Deployment](#final-model-deployment)
11. [Sample Prediction](#sample-prediction)

---



## <a id="imports--setup"></a>1. **Imports & Setup**

In [1]:
import os

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import joblib


## <a id="load--inspect-data"></a>2. **Load & Inspect Data**

In [2]:
data_path = "../data/region3_clean.csv"  # change to region3_clean.csv if needed

df = pd.read_csv(data_path)

print("Shape:", df.shape)
print(df.head())
print(df.isna().sum())


Shape: (1488, 299)
       # Date and time  wind_speed  Wind speed, Standard deviation (m/s)  \
0  2021-01-11 06:00:00   10.236168                              1.224991   
1  2021-01-11 08:10:00   10.186494                              1.233780   
2  2021-01-11 09:50:00   10.226459                              1.425780   
3  2021-01-11 10:00:00   10.452997                              1.175044   
4  2021-01-11 11:30:00   10.478650                              1.324442   

   Wind speed, Minimum (m/s)  Wind speed, Maximum (m/s)  Long Term Wind (m/s)  \
0                   8.094797                  11.770544                   7.1   
1                   8.086259                  12.148848                   7.1   
2                   6.840344                  12.812271                   7.1   
3                   8.167192                  12.697185                   7.1   
4                   8.744857                  13.281160                   7.1   

   Wind speed Sensor 1 (m/s)  Wind sp

## <a id="feature-selection"></a>3. **Feature Selection**

In [3]:
FEATURES = ["wind_speed", "rotor_speed", "power"]
TARGET = "pitch"

X = df[FEATURES].values
y = df[TARGET].values

print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (1488, 3)
y shape: (1488,)


## <a id="trainvalidationtest-split"></a>4. **Train/Validation/Test Split**

In [4]:
# First split: Train (70%) vs Temp (30% = Val + Test)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    shuffle=True
)

# Second split: from Temp (30%) -> Val (15%) + Test (15%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.5,
    random_state=42,
    shuffle=True
)

print("Train shape:", X_train.shape, y_train.shape)
print("Val shape:  ", X_val.shape, y_val.shape)
print("Test shape: ", X_test.shape, y_test.shape)


Train shape: (1041, 3) (1041,)
Val shape:   (223, 3) (223,)
Test shape:  (224, 3) (224,)


## <a id="data-scaling"></a>5. **Data Scaling**



In [5]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)
X_test_scaled  = scaler.transform(X_test)

print("Train scaled mean:", X_train_scaled.mean(axis=0))
print("Train scaled std: ", X_train_scaled.std(axis=0))


Train scaled mean: [-3.57638990e-15  1.22569795e-14  3.41321058e-14]
Train scaled std:  [1. 1. 1.]


## <a id="model-training"></a>6. **Model Training**


In [6]:
mlp = MLPRegressor(
    hidden_layer_sizes=(64, 64),  # two hidden layers
    activation="relu",
    solver="adam",
    learning_rate_init=0.001,
    max_iter=500,
    random_state=42,
    early_stopping=False  # we'll use our own val set, not internal one
)

mlp.fit(X_train_scaled, y_train)

print("Training finished.")
print("Number of iterations:", mlp.n_iter_)
print("Final training loss:", mlp.loss_)


Training finished.
Number of iterations: 134
Final training loss: 0.23077197121724324



## <a id="model-evaluation"></a>7. **Model Evaluation**


In [7]:
def evaluate(model, X, y, name="set"):
    preds = model.predict(X)
    mae = mean_absolute_error(y, preds)
    mse = mean_squared_error(y, preds)
    rmse = np.sqrt(mse)
    r2 = r2_score(y, preds)
    
    print(f"{name} metrics:")
    print(f"  MAE : {mae:.3f}")
    print(f"  RMSE: {rmse:.3f}")
    print(f"  R^2 : {r2:.3f}")
    print("-" * 30)
    
    return {"MAE": mae, "RMSE": rmse, "R2": r2}

metrics_train = evaluate(mlp, X_train_scaled, y_train, "Train")
metrics_val   = evaluate(mlp, X_val_scaled,   y_val,   "Validation")
metrics_test  = evaluate(mlp, X_test_scaled,  y_test,  "Test")

metrics_df = pd.DataFrame(
    [metrics_train, metrics_val, metrics_test],
    index=["Train", "Validation", "Test"]
)
metrics_df


Train metrics:
  MAE : 0.536
  RMSE: 0.670
  R^2 : 0.957
------------------------------
Validation metrics:
  MAE : 0.550
  RMSE: 0.681
  R^2 : 0.953
------------------------------
Test metrics:
  MAE : 0.558
  RMSE: 0.694
  R^2 : 0.949
------------------------------


,MAE,RMSE,R2
Train,0.536424,0.670312,0.957453
Validation,0.549740,0.681349,0.953171
Test,0.557732,0.693551,0.948806



## <a id="save-model--scaler"></a>8. **Save Model & Scaler**


In [8]:
os.makedirs("../models", exist_ok=True)

scaler_path = "../models/standard_scaler.joblib"
model_path  = "../models/mlp_pitch_regressor.joblib"

joblib.dump(scaler, scaler_path)
joblib.dump(mlp, model_path)

print("Saved scaler to:", scaler_path)
print("Saved model  to:", model_path)


Saved scaler to: ../models/standard_scaler.joblib
Saved model  to: ../models/mlp_pitch_regressor.joblib



## <a id="hyperparameter-tuning"></a>9. **Hyperparameter Tuning**


In [9]:
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    "hidden_layer_sizes": [(32, 32), (64, 64), (64, 64, 32), (128, 64)],
    "alpha": [1e-5, 1e-4, 1e-3, 1e-2],
    "learning_rate_init": [1e-3, 5e-4, 1e-4]
}

base_mlp = MLPRegressor(
    activation="relu",
    solver="adam",
    max_iter=400,
    random_state=42,
    early_stopping=False
)

rand_search = RandomizedSearchCV(
    base_mlp,
    param_distributions=param_dist,
    n_iter=10,
    cv=3,
    scoring="neg_mean_absolute_error",
    random_state=42,
    n_jobs=-1,
    verbose=2
)

rand_search.fit(X_train_scaled, y_train)

print("Best params:", rand_search.best_params_)
best_model = rand_search.best_estimator_

_ = evaluate(best_model, X_train_scaled, y_train, "Train (best)")
_ = evaluate(best_model, X_val_scaled,   y_val,   "Validation (best)")
_ = evaluate(best_model, X_test_scaled,  y_test,  "Test (best)")

# Optionally overwrite saved model with best one
joblib.dump(best_model, "../models/mlp_pitch_regressor_best.joblib")


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best params: {'learning_rate_init': 0.001, 'hidden_layer_sizes': (64, 64), 'alpha': 0.001}
Train (best) metrics:
  MAE : 0.534
  RMSE: 0.665
  R^2 : 0.958
------------------------------
Validation (best) metrics:
  MAE : 0.552
  RMSE: 0.684
  R^2 : 0.953
------------------------------
Test (best) metrics:
  MAE : 0.560
  RMSE: 0.694
  R^2 : 0.949
------------------------------


['../models/mlp_pitch_regressor_best.joblib']


## <a id="final-model-deployment"></a>10. **Final Model Deployment**


In [10]:
import os, joblib

os.makedirs("../models", exist_ok=True)

# Save the scaler used for X_train_scaled, X_val_scaled, X_test_scaled
joblib.dump(scaler, "../models/standard_scaler.joblib")

# Use the tuned best_model as the deployed model
joblib.dump(best_model, "../models/mlp_pitch_regressor.joblib")

print("Saved scaler + deployed model.")


Saved scaler + deployed model.



## <a id="sample-prediction"></a>11. **Sample Predictiong**

In [11]:
# Take one real sample from the test set
sample = X_test[0]
true_pitch = y_test[0]

print("Sample features:")
print("  wind_speed :", sample[0])
print("  rotor_speed:", sample[1])
print("  power      :", sample[2])
print("True pitch:", true_pitch)

pred_pitch = best_model.predict(scaler.transform([sample]))[0]
print("Model prediction:", pred_pitch)


Sample features:
  wind_speed : 10.76464195251465
  rotor_speed: 15.181466728448871
  power      : 1999.1320495605487
True pitch: 4.07300003618002
Model prediction: 3.8722298044444083
